In [1]:
library(GSVA)
library(GSEABase)
library(org.Hs.eg.db)

library(AnnotationDbi)
library(Biobase)
library(GSEABase)
library(GSVAdata)

library(genefilter)
library(limma)
library(RColorBrewer)

library(tidyverse)
library(msigdbr)
library(patchwork)
rm(list=ls())

library(stringr)

Loading required package: BiocGenerics


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, aperm, append, as.data.frame, basename, cbind,
    colnames, dirname, do.call, duplicated, eval, evalq, Filter, Find,
    get, grep, grepl, intersect, is.unsorted, lapply, Map, mapply,
    match, mget, order, paste, pmax, pmax.int, pmin, pmin.int,
    Position, rank, rbind, Reduce, rownames, sapply, setdiff, sort,
    table, tapply, union, unique, unsplit, which.max, which.min


Loading required package: Biobase

Welcome to Bioconductor

    Vignettes contain introductory material; view with
    'browseVignettes()'. To cite Bioconductor, see
    'citation("Biobase")', and for packages 'citation("pkgname")'.


Loading required package: annotate

Loading required package: AnnotationDbi

Loading required package: stats4

Loading required package: IRanges


# Load HALLMARK dataset

In [2]:
genesets <- msigdbr(species = "Homo sapiens", category = "H")
genesets = subset(genesets,select = c("gs_name","gene_symbol")) %>% as.data.frame()
library(stringr)
genesets$gs_name = str_remove(genesets$gs_name,"HALLMARK_")
genesets <- split(genesets$gene_symbol, genesets$gs_name )

In [3]:
names(genesets) 

[1] "ADIPOGENESIS"                      "ALLOGRAFT_REJECTION"              
 [3] "ANDROGEN_RESPONSE"                 "ANGIOGENESIS"                     
 [5] "APICAL_JUNCTION"                   "APICAL_SURFACE"                   
 [7] "APOPTOSIS"                         "BILE_ACID_METABOLISM"             
 [9] "CHOLESTEROL_HOMEOSTASIS"           "COAGULATION"                      
[11] "COMPLEMENT"                        "DNA_REPAIR"                       
[13] "E2F_TARGETS"                       "EPITHELIAL_MESENCHYMAL_TRANSITION"
[15] "ESTROGEN_RESPONSE_EARLY"           "ESTROGEN_RESPONSE_LATE"           
[17] "FATTY_ACID_METABOLISM"             "G2M_CHECKPOINT"                   
[19] "GLYCOLYSIS"                        "HEDGEHOG_SIGNALING"               
[21] "HEME_METABOLISM"                   "HYPOXIA"                          
[23] "IL2_STAT5_SIGNALING"               "IL6_JAK_STAT3_SIGNALING"          
[25] "INFLAMMATORY_RESPONSE"             "INTERFERON_ALPHA_RESPONSE"        
[27] "INTERFERON_GAMMA_RESPONSE"         "KRAS_SIGNALING_DN"                
[29] "KRAS_SIGNALING_UP"                 "MITOTIC_SPINDLE"                  
[31] "MTORC1_SIGNALING"                  "MYC_TARGETS_V1"                   
[33] "MYC_TARGETS_V2"                    "MYOGENESIS"                       
[35] "NOTCH_SIGNALING"                   "OXIDATIVE_PHOSPHORYLATION"        
[37] "P53_PATHWAY"                       "PANCREAS_BETA_CELLS"              
[39] "PEROXISOME"                        "PI3K_AKT_MTOR_SIGNALING"          
[41] "PROTEIN_SECRETION"                 "REACTIVE_OXYGEN_SPECIES_PATHWAY"  
[43] "SPERMATOGENESIS"                   "TGF_BETA_SIGNALING"               
[45] "TNFA_SIGNALING_VIA_NFKB"           "UNFOLDED_PROTEIN_RESPONSE"        
[47] "UV_RESPONSE_DN"                    "UV_RESPONSE_UP"                   
[49] "WNT_BETA_CATENIN_SIGNALING"        "XENOBIOTIC_METABOLISM"

# Construct expression matrix for GSVA

In [4]:
expr_path = '../../result/cci/IDC/trans_border_expr/'
GSVA_result_path = '../../result/cci/IDC/GSVA_result/'

### Our data: border_adata_expr_-4--8

In [5]:
border_4_8 = read.table(paste0(expr_path,'trans_border_adata_expr_-4--8.csv'),sep=",",header = TRUE)
head(border_4_8)

,gene,AAACCTCATGAAGTTG.1,AAACTGCTGGCTCCAA.1,AAATACCTATAAGCAT.1,AAATAGGGTGCTATTG.1,AACGATAATGCCGTAG.1,AAGTGTTTGGAGACGG.1,ACAAATCGCACCGAAT.1,ACAAGGAAATCCGCCC.1,ACAGGCTTGCCCGACT.1,⋯,TGGGCAGGCCACCGCA.1,TGGTCCCACGCTACGG.1,TGGTTCAACGGGTAAT.1,TGTACGAACAAATCCG.1,TGTGCTTTACGTAAGA.1,TTATTAGAGCGTGTTC.1,TTATTATCTGGAAGGC.1,TTCCGCGTGAGGCGAT.1,TTCTGACCGGGCTCAA.1,TTCTGCCGCGCCTAGA.1
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,AL627309.1,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0.0000000,0,0.0000000,0,0.0000000
2,AL627309.5,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0.0000000,0,0.0000000,0,0.0000000
3,LINC01409,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0.0000000,0,0.0000000,0,0.0000000
4,LINC01128,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0.9534882,0,0.7445193,0,0.9573873
5,LINC00115,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0.0000000,0,0.0000000,0,0.0000000
6,FAM41C,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0.0000000,0,0.0000000,0,0.0000000


In [6]:
raw_Expression = border_4_8[,-1]    
border_4_8 = border_4_8[rowSums(raw_Expression)>0,] 
rownames(border_4_8) = border_4_8$gene
border_4_8 = border_4_8[,-1]
border_4_8 = as.matrix(border_4_8)
head(border_4_8)

,AAACCTCATGAAGTTG.1,AAACTGCTGGCTCCAA.1,AAATACCTATAAGCAT.1,AAATAGGGTGCTATTG.1,AACGATAATGCCGTAG.1,AAGTGTTTGGAGACGG.1,ACAAATCGCACCGAAT.1,ACAAGGAAATCCGCCC.1,ACAGGCTTGCCCGACT.1,ACCGATATTTAATCAT.1,⋯,TGGGCAGGCCACCGCA.1,TGGTCCCACGCTACGG.1,TGGTTCAACGGGTAAT.1,TGTACGAACAAATCCG.1,TGTGCTTTACGTAAGA.1,TTATTAGAGCGTGTTC.1,TTATTATCTGGAAGGC.1,TTCCGCGTGAGGCGAT.1,TTCTGACCGGGCTCAA.1,TTCTGCCGCGCCTAGA.1
LINC01128,0,0,0.0000000,0.000000,0.0000000,0,0.0000000,0,0.0000000,0,⋯,0,0.000000,0,0,0.0000000,0.9534882,0,0.7445193,0,0.9573873
LINC02593,0,0,0.0000000,0.000000,0.0000000,0,0.0000000,0,0.0000000,0,⋯,0,0.000000,0,0,0.0000000,0.0000000,0,0.0000000,0,0.0000000
SAMD11,0,0,0.8491126,0.000000,0.0000000,0,0.0000000,0,0.0000000,0,⋯,0,0.000000,0,0,0.0000000,0.0000000,0,0.0000000,0,0.0000000
NOC2L,0,0,0.0000000,1.229893,1.2702346,0,0.5061383,0,0.5107411,0,⋯,0,1.223507,0,0,0.8283243,0.0000000,0,0.0000000,0,0.0000000
KLHL17,0,0,0.8491126,0.000000,0.0000000,0,0.0000000,0,0.0000000,0,⋯,0,0.000000,0,0,0.0000000,0.0000000,0,0.0000000,0,0.0000000
PLEKHN1,0,0,0.0000000,0.000000,0.8245456,0,0.0000000,0,0.5107411,0,⋯,0,0.000000,0,0,0.0000000,0.0000000,0,0.0000000,0,0.0000000


# GSVA 

In [7]:
gsva_border_4_8 <- gsva(border_4_8, 
                        genesets,
                        verbose=TRUE,
                        parallel.sz=10
                       )

Setting parallel calculations through a MulticoreParam back-end
with workers=10 and tasks=100.
Estimating GSVA scores for 50 gene sets.
Estimating ECDFs with Gaussian kernels
Estimating ECDFs in parallel on 10 cores


iteration: 
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100




  |======================================================================| 100%



In [8]:
gsva_border_4_8

,AAACCTCATGAAGTTG.1,AAACTGCTGGCTCCAA.1,AAATACCTATAAGCAT.1,AAATAGGGTGCTATTG.1,AACGATAATGCCGTAG.1,AAGTGTTTGGAGACGG.1,ACAAATCGCACCGAAT.1,ACAAGGAAATCCGCCC.1,ACAGGCTTGCCCGACT.1,ACCGATATTTAATCAT.1,⋯,TGGGCAGGCCACCGCA.1,TGGTCCCACGCTACGG.1,TGGTTCAACGGGTAAT.1,TGTACGAACAAATCCG.1,TGTGCTTTACGTAAGA.1,TTATTAGAGCGTGTTC.1,TTATTATCTGGAAGGC.1,TTCCGCGTGAGGCGAT.1,TTCTGACCGGGCTCAA.1,TTCTGCCGCGCCTAGA.1
ADIPOGENESIS,-4.300262e-02,-0.109312511,-0.090677520,-0.121593384,-0.275698498,-0.25821684,-0.13120399,-0.122431074,-1.492369e-01,-0.15123684,⋯,-0.1328709122,-0.241727242,-0.26933621,-0.273044507,-0.191616682,-0.15870112,-0.134856876,-0.025926226,0.0008729347,-0.172997609
ALLOGRAFT_REJECTION,-3.182164e-01,-0.510464773,-0.495553064,-0.362623431,-0.402520149,-0.38370545,-0.56168413,-0.503632884,-5.277802e-01,-0.53413641,⋯,-0.5084939441,-0.349062050,-0.38409943,-0.507176823,-0.451842321,-0.40450293,-0.534160863,-0.518691860,-0.3735336032,-0.486761690
ANDROGEN_RESPONSE,7.186484e-02,-0.126351586,-0.215556086,-0.128996760,0.011432206,-0.11668237,-0.05628180,0.011609080,-1.157358e-01,-0.31262510,⋯,0.1945888090,0.092145092,-0.07164334,-0.181537694,-0.105860075,0.03843580,-0.148075314,-0.019213702,0.0517039467,-0.040423009
ANGIOGENESIS,3.829238e-01,-0.131257685,0.019947138,0.209742577,0.237791867,0.15520081,-0.47049323,-0.438371477,-3.404690e-01,-0.11276063,⋯,-0.2888483249,0.226265951,-0.00841675,-0.064606432,-0.032501928,-0.18399351,-0.266332335,-0.003223323,-0.0130985416,0.001003006
APICAL_JUNCTION,-1.712185e-01,-0.187624296,-0.164561361,-0.105436935,-0.121874249,-0.12774776,-0.36048106,-0.311279780,-2.323052e-01,-0.29062333,⋯,-0.3031228553,-0.157530292,-0.14722204,-0.230973474,-0.031878380,-0.23985436,-0.260116203,-0.075398416,-0.1218016084,-0.121624178
APICAL_SURFACE,1.205954e-01,0.049981231,-0.088826816,0.075308220,-0.311292700,-0.03227539,-0.30817721,-0.231838549,-3.426265e-01,-0.15638667,⋯,0.1944561190,-0.050815612,-0.13018602,-0.050195697,-0.016082030,-0.19411723,-0.039037921,0.043767551,0.1273512916,-0.280163226
APOPTOSIS,-7.680095e-02,-0.272338610,-0.213773444,-0.191942872,-0.119500247,-0.06014904,-0.41964287,-0.341882617,-1.279986e-01,-0.11502673,⋯,-0.2196867263,-0.194888511,-0.10718940,-0.151831503,-0.235587703,-0.19447719,-0.171105285,-0.229251925,-0.0111474764,-0.057764122
BILE_ACID_METABOLISM,-3.966583e-02,-0.119522296,-0.132838227,-0.063832820,0.059950012,-0.04656485,0.09202620,0.132656237,-1.109061e-01,-0.03053158,⋯,-0.1159325095,-0.075644556,-0.15605134,0.108697976,0.033879180,0.01265134,0.003121267,-0.012875055,0.1698558064,-0.095750831
CHOLESTEROL_HOMEOSTASIS,-2.731692e-01,-0.254836249,0.013445311,-0.131463022,0.035710535,-0.24465755,-0.17377093,-0.083659028,-4.326406e-02,0.09902759,⋯,-0.1285962374,0.059762506,-0.30130470,-0.072994636,-0.274584204,-0.16428650,0.209005933,-0.239481500,0.0119317726,-0.286312790
COAGULATION,-8.723830e-02,-0.109371762,-0.247579800,-0.039369213,-0.236399321,-0.10597401,-0.49069228,-0.280235182,-3.939215e-01,-0.32681409,⋯,-0.2902559533,-0.155694471,-0.04920062,-0.367624618,-0.219216844,-0.32156065,-0.322101123,-0.287092512,-0.2701549668,-0.049319099


In [9]:
gsva_border_4_8 <- data.frame(Pathway=rownames(gsva_border_4_8), gsva_border_4_8, check.names = F)
head(gsva_border_4_8)

,Pathway,AAACCTCATGAAGTTG.1,AAACTGCTGGCTCCAA.1,AAATACCTATAAGCAT.1,AAATAGGGTGCTATTG.1,AACGATAATGCCGTAG.1,AAGTGTTTGGAGACGG.1,ACAAATCGCACCGAAT.1,ACAAGGAAATCCGCCC.1,ACAGGCTTGCCCGACT.1,⋯,TGGGCAGGCCACCGCA.1,TGGTCCCACGCTACGG.1,TGGTTCAACGGGTAAT.1,TGTACGAACAAATCCG.1,TGTGCTTTACGTAAGA.1,TTATTAGAGCGTGTTC.1,TTATTATCTGGAAGGC.1,TTCCGCGTGAGGCGAT.1,TTCTGACCGGGCTCAA.1,TTCTGCCGCGCCTAGA.1
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
ADIPOGENESIS,ADIPOGENESIS,-0.04300262,-0.10931251,-0.09067752,-0.12159338,-0.27569850,-0.25821684,-0.1312040,-0.12243107,-0.1492369,⋯,-0.1328709,-0.24172724,-0.26933621,-0.27304451,-0.19161668,-0.1587011,-0.13485688,-0.025926226,0.0008729347,-0.172997609
ALLOGRAFT_REJECTION,ALLOGRAFT_REJECTION,-0.31821638,-0.51046477,-0.49555306,-0.36262343,-0.40252015,-0.38370545,-0.5616841,-0.50363288,-0.5277802,⋯,-0.5084939,-0.34906205,-0.38409943,-0.50717682,-0.45184232,-0.4045029,-0.53416086,-0.518691860,-0.3735336032,-0.486761690
ANDROGEN_RESPONSE,ANDROGEN_RESPONSE,0.07186484,-0.12635159,-0.21555609,-0.12899676,0.01143221,-0.11668237,-0.0562818,0.01160908,-0.1157358,⋯,0.1945888,0.09214509,-0.07164334,-0.18153769,-0.10586008,0.0384358,-0.14807531,-0.019213702,0.0517039467,-0.040423009
ANGIOGENESIS,ANGIOGENESIS,0.38292379,-0.13125769,0.01994714,0.20974258,0.23779187,0.15520081,-0.4704932,-0.43837148,-0.3404690,⋯,-0.2888483,0.22626595,-0.00841675,-0.06460643,-0.03250193,-0.1839935,-0.26633233,-0.003223323,-0.0130985416,0.001003006
APICAL_JUNCTION,APICAL_JUNCTION,-0.17121845,-0.18762430,-0.16456136,-0.10543693,-0.12187425,-0.12774776,-0.3604811,-0.31127978,-0.2323052,⋯,-0.3031229,-0.15753029,-0.14722204,-0.23097347,-0.03187838,-0.2398544,-0.26011620,-0.075398416,-0.1218016084,-0.121624178
APICAL_SURFACE,APICAL_SURFACE,0.12059544,0.04998123,-0.08882682,0.07530822,-0.31129270,-0.03227539,-0.3081772,-0.23183855,-0.3426265,⋯,0.1944561,-0.05081561,-0.13018602,-0.05019570,-0.01608203,-0.1941172,-0.03903792,0.043767551,0.1273512916,-0.280163226


In [10]:
write.table(gsva_border_4_8, 
            file=paste(GSVA_result_path,'IDC_GSVA_border_-4--8.csv',sep=""),
            quote=F,
            sep=',',
            row.names=F    
           )